<a href="https://colab.research.google.com/github/AEGriffith/dnd_generate/blob/main/Generate_5e_Spells.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Steps

1.   Save a shortcut of the D&D Gen Public to your drive at the top level.
2.   Select your options in the form underthe "Final Generate" section.
3.   Click "Run All"
4.   Mount Google Drive in second code box.
5.   Click Link







## **NOTE:**
As long as everything has been run once and you haven't had to reconnect, you only need to run the [Final Generate](https://colab.research.google.com/drive/1J_LyLjq5c8QHQeudenp79CbIufppWOdI#scrollTo=Tunqz94JWiq7) section to make changes and generate more spells.

# Google Drive and GSpread
Interaction Required

## Mount Google Drive
Required for loading models. Nothing will work if the drive is not mounted.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Authorize GSpread
Required to get random spell names and save generated spells.



In [ ]:
# google sheets auth
from google.colab import auth
auth.authenticate_user()

import gspread
import random
from oauth2client.client import GoogleCredentials
gc = gspread.authorize(GoogleCredentials.get_application_default())

# Load Remaining Packages
No interaction Needed


## General

In [ ]:
%%capture
!pip install transformers

In [ ]:
# We'll need these libraries to gather and shape the data.
import json
import torch
import random
import sys
import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from torch.utils.data import Dataset, DataLoader, SequentialSampler, RandomSampler
from itertools import compress
from math import sqrt

## Description Imports
Click the link and copy the code into the box.

In [ ]:
import requests
import torch.nn.functional as F

from tqdm.std import trange
from transformers import GPT2Tokenizer, GPT2LMHeadModel, GPT2Config

## Prediction Imports

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import ConfusionMatrixDisplay
from transformers import BartForSequenceClassification, BartTokenizer 
from transformers import RobertaForSequenceClassification, RobertaTokenizer, RobertaConfig

## Description

### Load Spell Description Generation Models

In [ ]:
# should be folder that has model information
desc_model = GPT2LMHeadModel.from_pretrained("/content/drive/MyDrive/D&D Gen Public/final_models/description")
desc_tokenizer = GPT2Tokenizer.from_pretrained("/content/drive/MyDrive/D&D Gen Public/final_models/description")

## Spell Predictions

In [ ]:
PROJECT_FOLDER = '/content/drive/MyDrive/D&D Gen Public/final_models'
MODEL_FOLDER = 'level'
device = 'cuda'

### Load Spell Level Prediction Models

In [ ]:
lvl_model = BartForSequenceClassification.from_pretrained(os.path.join(PROJECT_FOLDER, MODEL_FOLDER)).cuda()
lvl_tokenizer = BartTokenizer.from_pretrained('facebook/bart-large')

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.56k [00:00<?, ?B/s]

### Load Spell School Prediction Models

In [ ]:
# from transformers.utils.dummy_pt_objects import DistilBertPreTrainedModel
school_model = RobertaForSequenceClassification.from_pretrained('/content/drive/MyDrive/D&D Gen Public/final_models/school/')
school_tokenizer = RobertaTokenizer.from_pretrained('/content/drive/MyDrive/D&D Gen Public/final_models/school/')

# Generation and Prediction Functions
No interaction needed

## Description generation functions

In [ ]:
def generate(
model,
tokenizer,
prompt,
entry_count=10,
entry_length=1000,
top_p=0.8,
temperature=1.1,
):
  model = model.to('cuda')
  model.eval()




  generated_num = 0
  generated_list = []

  filter_value = -float("Inf")

  with torch.no_grad():

      for entry_idx in trange(entry_count):

          entry_finished = False

          generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)
          generated = generated.to('cuda')
          # Using top-p (nucleus sampling): https://github.com/huggingface/transformers/blob/master/examples/run_generation.py
          i = 0

          while i <= entry_length:
              stuck = 0
              outputs = model(generated, labels=generated)
              loss, logits = outputs[:2]
              logits = logits[:, -1, :] / (temperature if temperature > 0 else 1.0)

              sorted_logits, sorted_indices = torch.sort(logits, descending=True)
              cumulative_probs = torch.cumsum(
                  F.softmax(sorted_logits, dim=-1), dim=-1
              )

              sorted_indices_to_remove = cumulative_probs > top_p
              sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[
                  ..., :-1
              ].clone()
              sorted_indices_to_remove[..., 0] = 0

              indices_to_remove = sorted_indices[sorted_indices_to_remove]
              logits[:, indices_to_remove] = filter_value

              next_token = torch.multinomial(F.softmax(logits, dim=-1), num_samples=1)


              if next_token in tokenizer.encode("<|spell|>") or next_token in tokenizer.encode("<|name|>"):
                # print(tokenizer.decode(next_token))
                i = i-1
                # generated = generated
                if stuck >500:
                  entry_finished = True
                stuck = stuck+1
                continue
              else:
                generated = torch.cat((generated, next_token), dim=1)
                stuck = 0

              if next_token in tokenizer.encode("<|endoftext|>"):
                  entry_finished = True

              if entry_finished:

                  generated_num = generated_num + 1

                  generated = generated.to('cpu')

                  output_list = list(generated.squeeze().numpy())
                  output_text = tokenizer.decode(output_list)

                  generated_list.append(output_text)
                  break
          
          if not entry_finished:
              generated = generated.to('cpu')
              output_list = list(generated.squeeze().numpy())
              output_text = f"{tokenizer.decode(output_list)}<|endoftext|>" 
              generated_list.append(output_text)
              
  return generated_list

## Level prediction functions

In [ ]:
class LevelsDataset(Dataset):
    def __init__(self, df, max_length = 900):
        self.df = df
        self.tokenizer = lvl_tokenizer
        self.max_length = max_length 
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        # input=review, label=stars
        review = self.df.loc[idx, 'description']
        # labels are 0-indexed
        label = int(self.df.loc[idx, 'level']) 
        
        encoded = self.tokenizer(
            review,                      # review to encode
            add_special_tokens=True,
            max_length=self.max_length,  # Truncate all segments to max_length
            padding='max_length',        # pad all reviews with the [PAD] token to the max_length
            return_attention_mask=True,  # Construct attention masks.
            truncation=True
        )
        
        input_ids = encoded['input_ids']
        attn_mask = encoded['attention_mask']
        
        return {
            'input_ids': torch.tensor(input_ids),
            'attn_mask': torch.tensor(attn_mask), 
            'label': torch.tensor(label)
        }

In [ ]:
def get_single_level_prediction(spell_desc, model):

  df = pd.DataFrame()
  df['description'] = [spell_desc]
  df['level'] = ['0']

  dataset = LevelsDataset(df)

  TEST_BATCH_SIZE = 1
  NUM_WORKERS = 1

  test_params = {'batch_size': TEST_BATCH_SIZE,
              'shuffle': True,
              'num_workers': NUM_WORKERS}

  data_loader = DataLoader(dataset, **test_params)

  total_examples = len(df)
  predictions = np.zeros([total_examples], dtype=object)

  for batch, data in enumerate(data_loader):

    # Get the tokenization values.
    input_ids = data['input_ids'].to(device)
    mask = data['attn_mask'].to(device)

    # Make the prediction with the trained model.
    outputs = model(input_ids, mask)

    # Get the star rating.
    big_val, big_idx = torch.max(outputs[0].data, dim=1)
    level_predictions = (big_idx).cpu().numpy()

  return level_predictions[0]

## School Prediction Functions

In [ ]:
class Schools(Dataset):
    def __init__(self, dataframe):
        self.len = len(dataframe)
        self.data = dataframe
        self.tokenizer = school_tokenizer
        self.max_len = 512
        
    def __getitem__(self, index):
        desc = str(self.data.description[index])
        desc = " ".join(desc.split())
        inputs = self.tokenizer.encode_plus(
            desc,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_token_type_ids=True,
            truncation=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']

        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'targets': torch.tensor(self.data.encode_cat[index], dtype=torch.long)
        } 
    
    def __len__(self):
        return self.len

In [ ]:
def get_single_school_prediction(spell_desc, model):
 
  df = pd.DataFrame()
  df['description'] = [spell_desc]
  df['encode_cat'] = [1]

  dataset = Schools(df)


  TEST_BATCH_SIZE = 1
  NUM_WORKERS = 1

  test_params = {'batch_size': TEST_BATCH_SIZE,
              'shuffle': True,
              'num_workers': NUM_WORKERS}

  data_loader = DataLoader(dataset, **test_params)

  total_examples = len(df)
  predictions = np.zeros([total_examples], dtype=object)

  for batch, data in enumerate(data_loader,0):

    # Get the tokenization values.
    input_ids = data['ids'].to(device, dtype = torch.long)
    mask = data['mask'].to(device, dtype = torch.long)

    # Make the prediction with the trained model.
    outputs = model(input_ids, mask)

    # Get the star rating.
    big_val, big_idx = torch.max(outputs[0].data, dim=0)
    school_predictions = (big_val).cpu().numpy()


  return np.argmax(school_predictions)

In [ ]:
school_encoding = ['Transmutation' , 'Abjuration', 'Evocation', 'Illusion', 'Conjuration', 'Enchantment', 'Divination', 'Necromancy']

## Generate functions

In [ ]:
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

In [ ]:
def name_gen():
  

  worksheet = gc.open('generated spells').worksheet('spell names')
  gen_names=[]
  # Convert to a DataFrame and render.
  df = pd.DataFrame.from_dict(worksheet.get_all_records())
  for i,j in df.iterrows():
    gen_names.append(df['names'][i])

  num_gen_spells = len(df)

  spell_name = random.choice(gen_names)
  return spell_name

In [ ]:
def generate_spells(name, num_spells=1, print_spell=True, save_spell=True):
  
  for i in range(0,num_spells):
    if use_generated:
      name = name_gen()

    prompt = f"<|name|> {name} <|spell|>"
    desc_model.to('cuda')
    gen_list = generate(desc_model, desc_tokenizer, prompt)
    school_model.to('cuda')
    save_results = []
    
    for desc in gen_list:
      remove_end_tokens = desc[9:-13].strip()
      name, desc = remove_end_tokens.split('<|spell|>', 1)
      lvl_result = get_single_level_prediction(desc, lvl_model)
      school_results = get_single_school_prediction(desc, school_model)
      
      if print_spell:
        print(name)
        print(desc)
        print("level: ", lvl_result)
        print("school: ", school_encoding[school_results])
        print("\n\n\n")
        save_results.append(name)
        save_results.append(lvl_result)
        save_results.append(school_encoding[school_results])
        save_results.append(desc)

    
    if save_spell:
      worksheet = gc.open('generated spells').worksheet('spells')

      # Convert to a DataFrame and render.
      df = pd.DataFrame.from_dict(worksheet.get_all_records())

      num_gen_spells = len(df)

      cell_begin = 'A' + str(num_gen_spells + 2)
      cell_end = 'D' + str(num_gen_spells + len(gen_list))

      cell_list = worksheet.range(cell_begin + ':' + cell_end)

      n = 0
      for cell in cell_list:
        cell.value = str(save_results[n])
        n += 1

      print(cell_list)
      worksheet.update_cells(cell_list)

# Final Generate

In [ ]:
#@title Generate  {display-mode: "form"}
#@markdown Check this box to use a generated spell name
use_generated = True #@param {type:"boolean"}

#@markdown Number of spells you want to generate. Ten descriptions are generated for each spell. If *use_generated* is checked, will choose spell names randomly from the Google sheet each iteration. If using a custom spell name, will generate 10\**num_spells* descriptions for *spell_name*.
num_spells =  1#@param {type:"integer"}

print_spells = True #@param {type:"boolean"}
save_spells = True #@param {type:"boolean"}

#param = spell_name
# spell name text
#@markdown If you're not generating spell names, put a name here. (if use_generated is checked, this name won't matter)
spell_name = "Dread Hands" #@param {type:"string"}


generate_spells(name=spell_name, num_spells=num_spells, print_spell=print_spells, save_spell=save_spells)


100%|██████████| 10/10 [00:23<00:00,  2.36s/it]


Frostbreath 
 A strong wind howls around you in a 100-foot radius and moves with you, remaining centered on you. The wind lasts for the spell's duration or until you use your action to dismiss it.

You can have the wind pushed against you (your choice), rolling with it as you move. If you do, each creature in the area must make a Constitution saving throw. A creature takes 2d10 bludgeoning damage on a failed save, or half as much damage on a successful one.

A creature restrained by the wind is engulfed in frigid flames if it is not flying.
level:  3
school:  Evocation




Frostbreath 
 You freeze the air within a 15-foot cube you can see within range. Until the spell ends, freezing air in the cube moves at 50 feet per round for the duration. A creature in the cube must make a Dexterity saving throw. On a failed save, a creature takes 5d8 cold damage, or half as much damage on a successful save.
level:  3
school:  Evocation




Frostbreath 
 You exhale a blast of frigid air that deals 